In [1]:
import requests
import folium
import pandas as pd
import numpy as np
import json
import matplotlib.cm as cm
from pandas.io.json import json_normalize
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [2]:
## download data set

In [3]:
from pandas.io.json import json_normalize

In [4]:
dat=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [5]:
sp=BeautifulSoup(dat, 'html.parser')

In [6]:
pl=[]
bl=[]
nl=[]

In [7]:
sp.find('table').find_all('tr')
sp.find('table').find_all('tr')
for r in sp.find('table').find_all('tr'):
    cells=r.find_all('td')

In [8]:
for r in sp.find('table').find_all('tr'):
    cells=r.find_all('td')
    if(len(cells)>0):
        pl.append(cells[0].text.rstrip('\n'))
        bl.append(cells[1].text.rstrip('\n'))
        nl.append(cells[2].text.rstrip('\n'))

In [9]:
td=pd.DataFrame({"PostalCode":pl,
                "Borough":bl,
                "Neighborhood":nl})
td.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
tdr=td[td.Borough!="Not assigned"].reset_index(drop=True)
tdr.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
tdg=tdr.groupby(["PostalCode","Borough"],as_index=False).agg(lambda x:", ".join(x))

In [12]:
tdg.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
for index,r in tdg.iterrows():
    if r["Neighborhood"]=="Not assigned":
        r["Neighborhood"]==r["Borough"]
tdg.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
## question 1 requirement

In [15]:
cl=["PostalCode","Borough","Neighborhood"]
td=pd.DataFrame(columns=cl)

In [16]:
tl=["M5G","M2H","M4B","M5A"]
for pc in tl:
    td=td.append(tdg[tdg["PostalCode"]==pc],ignore_index=True)
td

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [17]:
df=tdg

In [18]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [19]:
print(df.shape)

(103, 3)


In [20]:
df.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M4L,North York,Downsview
freq,1,24,4


In [21]:
import geocoder
def get_latlong(postalcode):
    lati_long_coords=None
    while(lati_long_coords is None):
        g=geocoder.arcgis('{},Toronto.Ontario'.format(postalcode))
        lati_long_coords=g.latlng
    return lati_long_coords
get_latlong('M4G')

[43.70909000000006, -79.36409999999995]

In [22]:
postalcod=df['PostalCode']
cods=[get_latlong(postalcode) for postalcode in postalcod.tolist()]

In [23]:
dfc=pd.DataFrame(cods,columns=['Latitude','Longitude'])
df['Latitude']=dfc['Latitude']
df['Longitude']=dfc['Longitude']

In [24]:
df[df.PostalCode=='M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493


In [25]:
df.head(15)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944
5,M1J,Scarborough,Scarborough Village,43.74309,-79.23526
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.72861,-79.26367
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.71406,-79.28412
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.72360,-79.23496
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.69539,-79.26194


In [27]:
df.to_csv('toronto2.csv',index=False)